Obtengo los datos de la API y los visualizo con PANDAS

In [1]:
import requests

import pandas as pd
from datetime import date

import psycopg2
from dotenv import load_dotenv
import os 

from psycopg2.extras import execute_values

url = "https://api.coinlore.net/api/tickers/?start=0&limit=100"
response = requests.get(url)
datos= response.json()



df=pd.DataFrame.from_dict(datos['data']).reset_index().rename(columns={"index":'indice'})
df


,indice,id,symbol,name,nameid,rank,price_usd,percent_change_24h,percent_change_1h,percent_change_7d,price_btc,market_cap_usd,volume24,volume24a,csupply,tsupply,msupply
0,0,90,BTC,Bitcoin,bitcoin,1,72457.40,-0.44,-0.23,9.64,1.00,1423309010688.00,4.384159e+10,5.974106e+10,19643390.00,19643390,21000000
1,1,80,ETH,Ethereum,ethereum,2,3924.48,-1.97,-0.24,3.06,0.054130,480259994757.31,1.874480e+10,2.338137e+10,122375302.00,122375302,
2,2,2710,BNB,Binance Coin,binance-coin,3,596.41,0.83,-1.12,39.19,0.008226,99482033549.41,3.081419e+09,3.523301e+09,166801148.00,192443301,200000000
3,3,518,USDT,Tether,tether,4,1.00,-0.08,0.08,-0.22,0.000014,90848864851.44,8.288420e+10,1.007549e+11,90653779562.00,93609080552.592,
4,4,48543,SOL,Solana,solana,5,169.26,10.56,-0.95,26.69,0.002335,72175249739.13,5.837607e+09,4.305130e+09,426405733.00,491561409.22682,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,95,99329,WLD,Worldcoin,worldcoin-wld,96,9.55,-2.37,0.12,23.30,0.000132,1056086601.17,4.088710e+08,4.852916e+08,110601455.00,10000000000,10000000000
96,96,99333,AXL,Axelar,axelar,97,2.10,-4.01,-0.68,-4.66,0.000029,1051566692.17,4.721650e+06,5.211267e+06,501601923.00,1079242404,None
97,97,2729,ZRX,0x,0x,98,1.23,29.58,0.54,140.57,0.000017,1043394535.17,4.557391e+08,3.002988e+08,847496054.00,1000000000,1000000000
98,98,48537,LUNC,Terra Classic,terra-luna,99,0.000178,-2.89,-0.94,-8.62,2.46E-9,1038656166.88,2.502688e+07,3.261974e+07,5832569595040.00,6576804998885.5,6576804998885.5


In [11]:
hora_actual = pd.to_datetime('now')
hora_formateada = hora_actual.strftime("%D %H:%M:%S")


Aplico un filtro de los datos que me interesan y les agrego la hora de Extaccion 

In [13]:

df_filtrado= df.loc[0:15,['indice','id','symbol','name','rank','price_usd','percent_change_24h','percent_change_1h','market_cap_usd']]

hora_actual = pd.to_datetime('now')
hora_formateada = hora_actual.strftime("%D %H:%M:%S")

df_filtrado =df_filtrado.assign(Extraction_time =hora_formateada)
df_filtrado

,indice,id,symbol,name,rank,price_usd,percent_change_24h,percent_change_1h,market_cap_usd,Extraction_time
0,0,90,BTC,Bitcoin,1,72457.40,-0.44,-0.23,1423309010688.00,03/14/24 10:41:04
1,1,80,ETH,Ethereum,2,3924.48,-1.97,-0.24,480259994757.31,03/14/24 10:41:04
2,2,2710,BNB,Binance Coin,3,596.41,0.83,-1.12,99482033549.41,03/14/24 10:41:04
3,3,518,USDT,Tether,4,1.00,-0.08,0.08,90848864851.44,03/14/24 10:41:04
4,4,48543,SOL,Solana,5,169.26,10.56,-0.95,72175249739.13,03/14/24 10:41:04
5,5,46971,STETH,Staked Ether,6,3913.40,-1.84,-0.24,36082338830.51,03/14/24 10:41:04
6,6,58,XRP,XRP,7,0.680910,-0.82,-0.40,29217516600.32,03/14/24 10:41:04
7,7,257,ADA,Cardano,8,0.777207,2.74,-1.25,27456886757.19,03/14/24 10:41:04
8,8,2,DOGE,Dogecoin,9,0.186757,10.16,2.14,26764494313.27,03/14/24 10:41:04
9,9,33285,USDC,USD Coin,10,0.999401,0.00,0.00,24055198073.43,03/14/24 10:41:04


Cambio los tipos de datos

In [3]:
df_filtrado["price_usd"]= df_filtrado["price_usd"].astype("float64")

df_filtrado["percent_change_24h"]= df_filtrado["percent_change_24h"].astype("float64")

df_filtrado["percent_change_1h"]= df_filtrado["percent_change_1h"].astype("float64")

df_filtrado["market_cap_usd"]= df_filtrado["market_cap_usd"].astype("float64")

Conexion con Amazon redshift

In [4]:
#Prueba de conexion
load_dotenv()

"""url = os.getenv('url-sql')
data_base= os.getenv('data_base')
user=os.getenv('user')
pwd=os.getenv('pwd')"""
url= "data-engineer-cluster.cyhh5bfevlmn.us-east-1.redshift.amazonaws.com"
data_base='data-engineer-database'
user='luchitrading_coderhouse'
pwd='6vJ8t1V7Tp'


try:
    conn = psycopg2.connect(
        host=url,
        dbname=data_base,
        user=user,
        password=pwd,
        port='5439'
    )
    print("Conectado a Postgres")
    
except Exception as e:
    print("No es posible conectarse a Postgres")
    print(e)

Conectado a Postgres


Creacion de tabla en redshift

In [5]:
def cargar_en_postgres(conn, table_name, dataframe):
    dtypes= dataframe.dtypes
    cols= list(dtypes.index )
    tipos= list(dtypes.values)
    type_map = {'int64': 'INT','int32': 'INT','float64': 'FLOAT','object': 'VARCHAR(50)','bool':'BOOLEAN'}
    sql_dtypes = [type_map[str(dtypes)] for dtypes in tipos]
    # Definir formato SQL VARIABLE TIPO_DATO
    column_defs = [f"{name} {data_type}" for name, data_type in zip(cols, sql_dtypes)]
    # Combine column definitions into the CREATE TABLE statement
    table_schema = f"""
        CREATE TABLE IF NOT EXISTS {table_name} (
            {', '.join(column_defs)}
        );
        """
    # Crear la tabla
    cur = conn.cursor()
    cur.execute(table_schema)
    # Generar los valores a insertar
    values = [tuple(x) for x in dataframe.to_numpy()]
    # Definir el INSERT
    insert_sql = f"INSERT INTO {table_name} ({', '.join(cols)}) VALUES %s"
    # Execute the transaction to insert the data
    cur.execute("BEGIN")
    execute_values(cur, insert_sql, values)
    cur.execute("COMMIT")
    print('Proceso terminado')

In [6]:
cargar_en_postgres(conn=conn, table_name= "ValoresCryptos", dataframe=df_filtrado)

Proceso terminado
